<a href="https://colab.research.google.com/github/Yasaman-habibi/Pre_Processing_Report/blob/main/Report.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Import Library
import os
import re
import nltk
import glob
import pandas as pd
from time import sleep
import matplotlib.pyplot as plt
from google.colab import drive, files
from google.colab import drive
import ipywidgets as widgets
import seaborn as sns
import numpy as np

In [ ]:
#Upload Files

drive.mount('/content/drive')
source_path = "/content/drive/MyDrive/Combined_Texts"
all_txt_files = glob.glob(os.path.join(source_path, "*.txt"))

selector = widgets.SelectMultiple(
    options=all_txt_files,
    description='Select files',
    rows=10
)
display(selector)

In [ ]:
uploaded_Texts = list(selector.value)

In [ ]:
Cleaned_path = "/content/drive/MyDrive/sustainability_table/PivotReport"
os.makedirs(Cleaned_path, exist_ok=True)
print(" انتخاب شد:\n" + "\n".join(uploaded_Texts))

In [ ]:
keywords_path = "/content/drive/MyDrive/sustainability_table/PivotReport"
os.makedirs(keywords_path, exist_ok=True)
keywords_file = os.path.join(keywords_path, "keywords.xlsx")

# KeyWords for Report
default_keywords = [
     "Sustainable" , "Sustainability" , "Sustainable finance" , "Sustainable innovation" , "Sustainable agriculture" ,
     "Sustainable materials" ,  "Sustainable supply chain" , "Sustainable development" ,
     "SDGs" , "ESG", "Non-Financial" , "Development" ,
     "Environmental" ,  "Environmental, social, and governance" , "Environmental protection", "Environmental impact" , "environment",
     "climate" , "Climate mitigation" , "Climate change" ,
     "Economic" , "Economic sustainability" , "Economy" , "Circular economy" , "Green economy" , "Green technology" ,
     "Social" , "Society" , "Corporate social responsibility" ,"CSR" ,
     "Carbon footprint", "Carbon emissions" ,  "Pollutants " ,  "Greenhouse Gas Emissions" ,  "Decarbonization" ,
     "Renewable energy" , "Clean energy" , "Energy efficiency" , "Recycling" , "Demographic changes" ,
     "Waste management" , "Zero waste" , "Natural Resources" , "Resource management" ,
     "Earth", "Air", "biodiversity",  "Crisis" ,  "Atmospheric" , "Water" , "pollution" , "Pollution reduction" ,
     "Drought" , "Famine" ,  "Water conservation" ,
     "Ground Warming" , "Global Warming" , "Species extinction" , "Ecosystem preservation" ,
     "Future Needs" ,  "Life cycle assessment" , "Eco-friendly" , "Responsible consumption" , "Human rights" ,
     "Better life",
]
updated_keywords_df = pd.DataFrame(sorted(set(default_keywords)), columns=["keyword"])
updated_keywords_df.to_excel(keywords_file, index=False)
keywords = updated_keywords_df["keyword"].str.lower().tolist()

In [ ]:
#Function

def process_reports_tagged(uploaded_Texts, sustain_terms, report_path):
    records = []

    sustain_terms = sorted(set([kw.strip().lower() for kw in sustain_terms]),
                           key=lambda x: len(x.split()), reverse=True)

    for file_path in uploaded_Texts:
        with open(file_path, "r", encoding="utf-8") as f:
            content = f.read()

        pattern = r"===== Start of File: (.+?) =====\n(.*?)\n===== End of File: \1 ====="
        reports = re.findall(pattern, content, flags=re.DOTALL)

        for report_name, report_text in reports:
            text = report_text.lower()
            counts = {}

            for term in sustain_terms:
                term_pattern = r"\b" + re.escape(term) + r"\b"
                matches = list(re.finditer(term_pattern, text))
                if matches:
                    counts[term] = len(matches)
                    text = re.sub(term_pattern, " " * len(term), text)

            for kw, cnt in counts.items():
                records.append({"report_name": report_name, "keyword": kw, "count": cnt})

    # Create DataFrame
    new_df = pd.DataFrame(records)

    if os.path.exists(report_path):
        old_df = pd.read_excel(report_path)
        combined_df = pd.concat([old_df, new_df], ignore_index=True)
    else:
        combined_df = new_df

    # Remove duplicate
    combined_df["report_name"] = combined_df["report_name"].str.replace(r"\s*\(\d+\)$", "", regex=True)
    combined_df = combined_df.drop_duplicates()


    # Pivot Table
    pivot_df = combined_df.pivot_table(
        index="report_name",
        columns="keyword",
        values="count",
        aggfunc="sum",
        fill_value=0
    )

    pivot_df.columns = pivot_df.columns.astype(str)
    pivot_df = pivot_df[sorted(pivot_df.columns)]

    pivot_df.to_excel(report_path)
    print(f" گزارش pivot ذخیره شد در: {report_path}")
    return pivot_df

In [ ]:
sustain_Dic = pd.read_excel("/content/drive/MyDrive/sustainability_table/PivotReport/keywords.xlsx")
keywords = sustain_Dic["keyword"].dropna().str.strip().tolist()

report_path = "/content/drive/MyDrive/sustainability_table/PivotReport/report_pivot.xlsx"

In [ ]:
pivot_df = process_reports_tagged(uploaded_Texts, keywords, report_path)

pivot_df